In [ ]:
!python -m spacy download en_core_web_lg  #spacy词形还原模型，首次执行后需重新启动代码执行程序

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180944 sha256=715a340435d7f8587a65d853fbea08f8dc89799b3d2e7e95d3862330fea0c381
  Stored in directory: /tmp/pip-ephem-wheel-cache-ea4gyj20/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
import re
import nltk
import time
import pandas as pd

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# 去停用词
remove_list = ['but', 'few', 'no', 'nor', 'not', 'don', "don't", 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn',
        "didn't", "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
        'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", "shan't", 'shouldn', "shouldn't", 'wasn',
        "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"] #将这些词从停用词表中删除
stop_words = set(stopwords.words('english'))
stop_words.update('u','rt') #增加两个停用词：rt和u

for word in remove_list:
  stop_words.remove(word)

def remove_urls(s): #去超链接
    # re.sub(pattern,repl,string) is used to replace substrings. Will replace the matches in string with repl
    return re.sub(r"https?://\S+", "", s)

def clean_txt(input_text):
    input_text = str(input_text)
    if type(input_text) != str:
        return input_text
    # removing hashtags,emojis,hyperlinks
    input_text = re.sub(r"#+", "", str(input_text)) #只去掉'#'号
    input_text = input_text.encode("ascii", "ignore") #去ASCII码（表情）
    input_text = input_text.decode()
    input_text = remove_urls(input_text)   #去超链接     
    input_text = re.sub("amp", "", input_text) #去掉'amp'
    input_text = re.sub("nan", "", input_text) #去掉'nan'

    # removing @user
    r = re.findall(r"@[\w]*", input_text)
    for i in r:
      input_text = re.sub(i, "", input_text)

    input_text = " ".join([i for i in wordpunct_tokenize(input_text) if not i.lower() in stop_words]) #如果某单词的小写形式在停用词表中，则去掉该词

    # 去标点符号
    result = re.sub(r"[^\w\s]", "", input_text)
    
    # removing links
    #rresultes = re.sub(r"https[\w]*", "", result, flags=re.MULTILINE)

    # 去数字
    result = "".join(i for i in result if not i.isdigit())
    
    # 去空格
    result = re.sub("\s+", " ", result)
    if result[0] == " ":
      result = result[1:]
    if result[-1] == " ":
      result = result[:-1]
      
    return result

In [ ]:
def lemmatize_sentence_nltk(sentence):  #用NLTK进行词形还原（效果不佳，弃用）
  wnl=WordNetLemmatizer()
  new_sentence = []
  for word,tag in pos_tag(nltk.word_tokenize(sentence)): #分词+词性标注
    lemmatized = ""
    if word.istitle():
      if tag.startswith('NN'):
          lemmatized = wnl.lemmatize(word.lower(), pos='n')
      elif tag.startswith('VB'):
          lemmatized = wnl.lemmatize(word.lower(), pos='v')
      elif tag.startswith('JJ'):
          lemmatized = wnl.lemmatize(word.lower(), pos='a')
      elif tag.startswith('R'):
          lemmatized = wnl.lemmatize(word.lower(), pos='r')
      else:
          lemmatized = word

      lemmatized = lemmatized.capitalize()
    else:
      if tag.startswith('NN'):
          lemmatized = wnl.lemmatize(word, pos='n')
      elif tag.startswith('VB'):
          lemmatized = wnl.lemmatize(word, pos='v')
      elif tag.startswith('JJ'):
          lemmatized = wnl.lemmatize(word, pos='a')
      elif tag.startswith('R'):
          lemmatized = wnl.lemmatize(word, pos='r')
      else:
          lemmatized = word

    new_sentence.append(lemmatized)

  new_sentence = " ".join(new_sentence)
  return new_sentence

In [ ]:
def get_cleaned_data(df): #数据清理
  t1 = time.time()
  print('正在进行数据清理...')
  df['content'] = df.content.apply(lambda x:clean_txt(x))
  t2 = time.time()
  print('数据清理耗时',t2-t1,'s')

  return df

In [ ]:
from joblib import Parallel, delayed #用joblib加速词形还原

def lemmatize_pipe(doc):
  lemma_list = [tok.lemma_.upper() if tok.is_upper else tok.lemma_ for tok in doc] 

  return lemma_list

def chunker(iterable, total_length, chunksize):
  return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

def flatten(list_of_lists):
  "Flatten a list of lists to a combined list"
  return [item for sublist in list_of_lists for item in sublist]

def process_chunk(texts):
  preproc_pipe = []
  for doc in nlp.pipe(texts, batch_size=128): #用spacy.pipe加速
      preproc_pipe.append(lemmatize_pipe(doc))
  return preproc_pipe

def preprocess_parallel(texts, chunksize=1000): #加速
  executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
  do = delayed(process_chunk)
  tasks = (do(chunk) for chunk in chunker(texts, len(texts), chunksize=chunksize))
  result = executor(tasks)
  return flatten(result)

def lemmatize_sentence_spacy(df):  #用spaCy进行词形还原
  t1 = time.time()
  print('正在进行词形还原...')

  docs = preprocess_parallel(df.content, chunksize=1000)
  lemmatized_docs = []

  for sentence in docs:
    lemmatized = " ".join(sentence)
    lemmatized_docs.append(lemmatized)

  df['content'] = lemmatized_docs

  t2 = time.time()
  print('词形还原耗时',t2-t1,'s')

  return df

In [ ]:
def extract_data(df): #数据归约，从原始数据中提取有用数据
  columns1 = ['date','content','user','replyCount','retweetCount','likeCount','quoteCount'] #日期与时间，文本内容，用户信息，回复数，转发数，点赞数，引用数
  
  t1 = time.time()
  print('正在进行数据归约...')
  
  df = df[columns1].copy()
 
  df['length']=df.content.apply(lambda x:len(x))      #文本长度
  df['date']=df.date.apply(lambda x:x[:10])         #日期
  df['year']=df.date.apply(lambda x:x.split("-")[0])     #年份
  df['month']=df.date.apply(lambda x:x.split("-")[1])    #月份
  df['day']=df.date.apply(lambda x:x.split("-")[2])     #日
  df['username']=df.user.apply(lambda x:eval(x)['username']) #用户名
  df['location']=df.user.apply(lambda x:eval(x)['location']) #地点(不一定精确)
 
  t2 = time.time()
  print('数据归约耗时',t2-t1,'s')

  columns2 = ['date','year','month','day','content','length','replyCount','retweetCount','likeCount','quoteCount','username','location']
  
  return df[columns2]

In [ ]:
def get_preprocessed_data(path):
  print('文件路径：',path)
  df = pd.read_pickle(path)
  df = df.drop_duplicates(subset=['content'],keep='first',ignore_index=True) #文本去重
  df = get_cleaned_data(df)
  df = lemmatize_sentence_spacy(df)
  df = extract_data(df)

  return df

In [ ]:
df = get_preprocessed_data("/content/drive/My Drive/202012.pkl") #读取pkl格式文件，读取速度更快

文件路径： /content/drive/My Drive/202012.pkl
正在进行数据清理...
数据清理耗时 306.22693848609924 s
正在进行词形还原...
词形还原耗时 2445.876158952713 s
正在进行数据归约...
数据归约耗时 157.14918875694275 s


In [ ]:
df.sort_values(by='date',inplace=True,ignore_index=True) #按日期排序，并重建行索引

In [ ]:
df

,date,year,month,day,content,length,replyCount,retweetCount,likeCount,quoteCount,username,location
0,2020-12-01,2020,12,01,do bit keep patient staff safe well tuesdayfri...,59,0,0,5,0,gabyvparker,
1,2020-12-01,2020,12,01,Battle Axe Division cover sixth leg Konark Cor...,234,8,197,848,3,IaSouthern,"Pune, India"
2,2020-12-01,2020,12,01,pandemic happen regularly afford pretend don C...,66,0,2,0,0,taggcleanhands,"Toronto, Ontario"
3,2020-12-01,2020,12,01,read entrail use wastewater epidemiology track...,93,1,0,2,0,RSCTheAcademies,Ottawa - Canada
4,2020-12-01,2020,12,01,RGGI Next TSNP News Today Resgreen Group compl...,216,0,2,3,0,stockpickmover,
...,...,...,...,...,...,...,...,...,...,...,...,...
1112415,2020-12-31,2020,12,31,beat track also report pandemic force large nu...,165,1,0,0,0,soutikBBC,BBC News
1112416,2020-12-31,2020,12,31,June tell story miracle recovery Nitaidas Mukh...,165,1,0,0,0,soutikBBC,BBC News
1112417,2020-12-31,2020,12,31,July write whether India enough oxygen supply ...,123,1,0,0,0,soutikBBC,BBC News
1112418,2020-12-31,2020,12,31,avoid zombification COVID consumption game cha...,50,0,1,4,0,rdomenechv,Spain


In [ ]:
df.to_pickle('/content/drive/My Drive/202012_processed.pkl') #保存为pkl格式文件

In [ ]:
x.content[1]

'Lower than vermin, @BBCNews @BBCNewsnight @BBCPolitics @BBCr4today @BBCWorldatOne @BBCPM @Channel4News #QED #ToryScum #bbcaq #ToryCovidCatastrophe #COVID19 #murderers #LabourLeaks #LabourLeadershipElection2021 #FordeReport #LabourScum #Fascist #ApartheidIsrael  #StarmerOut https://t.co/mrIKMoifTA'

In [ ]:
df.content[1]

'low vermin night QED ToryScum bbcaq torycovidcatastrophe COVID murderer LabourLeaks LabourLeadershipElection FordeReport LabourScum Fascist ApartheidIsrael StarmerOut'